In [2]:
import data
import eval
import model
import preprocess
import postprocess

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

ONLY RUN IF THERE IS NEW DATA, TAKES LONG, NEXT CELL LOADS DATA FROM CSV AND PKL FILE

First off, the steps taken to fully load in the data set.

Step 1: load the raw text files and their id's into a dataframe

Step 2: if the texts are not in English, translate them

Step 3: concatenate the text files into a single dataset

Step 4: load in the labels

Step 5: merge the text and labels data sets

This can take quite a while, due to the translation step, skip this for the next cell, once we have all the data correctly saved as a csv, so we just load in that csv again. 

In [50]:
# load english text data
df_text_en = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/raw-documents")

# load and translate hindi and portugese text - part that takes long
df_text_bg = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/raw-documents", True, source_lang ="bg")
df_text_hi = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/raw-documents", True, source_lang="hi")
df_text_pt = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/raw-documents", True, source_lang = "pt")
df_text = pd.concat([df_text_en, df_text_bg, df_text_hi, df_text_pt])

# load all the labels, these are all in english
df_labels_en = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-2-annotations.txt")
df_labels_bg = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/subtask-2-annotations.txt")
df_labels_hi = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-2-annotations.txt")
df_labels_pt = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-2-annotations.txt")
df_labels = pd.concat([df_labels_en, df_labels_bg, df_labels_hi, df_labels_pt])


dom_mlb, sub_mlb, dom_narr_enc, sub_narr_enc, df = preprocess.encode_labels(pd.merge(df_text, df_labels, on="id"))

# save the mlb's, and the dataframe, so we can read them instead of processing the entire data again
with open("dom_mlb.pkl", "wb") as f:
    pickle.dump(dom_mlb,f)

with open("sub_mlb.pkl", "wb") as f:
    pickle.dump(sub_mlb,f)

df.to_csv("data.csv")

Data length:  200
Error while reading:BG_828.txt
Error while reading:BG_551.txt
Error while reading:BG_1559.txt
Error while reading:BG_630.txt
Error while reading:BG_839.txt
Error while reading:BG_971.txt
Error while reading:BG_1164.txt
Error while reading:BG_1249.txt
Error while reading:BG_526.txt
Error while reading:BG_454.txt
Error while reading:BG_521.txt
Error while reading:BG_509.txt
Error while reading:BG_513.txt
Error while reading:BG_1725.txt
Error while reading:BG_894.txt
Error while reading:BG_663.txt
Error while reading:BG_439.txt
Error while reading:BG_613.txt
Error while reading:BG_607.txt
Error while reading:BG_1345.txt
Error while reading:BG_171.txt
Error while reading:BG_561.txt
Error while reading:BG_548.txt
Error while reading:BG_1637.txt
Data length:  187
Error while reading:HI_82.txt
Error while reading:HI_55.txt
Error while reading:HI_115.txt
Error while reading:HI_128.txt
Error while reading:HI_100.txt
Error while reading:HI_40.txt
Error while reading:HI_54.txt
E

In [3]:
explanation_df = data.load_explanation_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-3-annotations.txt")
explanation_df.to_clipboard()


Article quoting Ukrainian officials, and indirectly arguing that situation in Ukraine is hopeless with the West having doubts in Ukraine's success. Ukrainian contradictions are highlighted in order to frame Ukraine as having no real options.
The text has several paragraphs underlining the potential use of nuclear weapons by Russia both in the context of Ukraine war as well as in a global scenario.
The text presents several paragraphs underlining that Wester countries helping Ukraine and NATO have the moral authority to help Ukraine to fight Russia.
The article talks about climate activists attacking artworks in Norway to protest oil and gas supply from Norway to Europe. The author uses keywords such as "vandals", "barbarians", and "fanatics" to describe those activists.
The text criticises the West and the US using several arguments, among which the corruption of Western elites as well as the system that is incompatible with freedom and thus support for Russia is the conflict is justif

first load the data set, and save it as a csv?

In [51]:
df = pd.read_csv("data.csv")

# load the mlb files back in, to get the classes and transform functions
with open("dom_mlb.pkl", "rb") as f:
    dom_mlb = pickle.load(f)

with open("sub_mlb.pkl", "rb") as f:
    sub_mlb = pickle.load(f)

print(len(df))
print(df.head())
print(dom_mlb.classes_)
print(sub_mlb.classes_)

626
   Unnamed: 0                id  \
0           0  EN_UA_104876.txt   
1           1  EN_UA_023211.txt   
2           2  EN_UA_011260.txt   
3           3  EN_UA_101067.txt   
4           4  EN_UA_102963.txt   

                                                text  \
0  Putin honours army unit blamed for Bucha massa...   
1  Europe Putin thanks US journalist Tucker Carls...   
2  Russia has a clear plan to resolve the conflic...   
3  First war of TikTok era sees tragedy, humor an...   
4  Ukraine's President Zelenskyy to address Mexic...   

                                            dom_narr  \
0                                          ['Other']   
1                                          ['Other']   
2  ['URW: Russia is the Victim', 'URW: Discrediti...   
3                                          ['Other']   
4                                          ['Other']   

                                            sub_narr  \
0                                          ['Other']   

Train it on the sub classes

In [52]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)
y = df[sub_mlb.classes_].values # omg this works
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [53]:
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=sub_mlb.classes_))

Skipped fold 51, labels invalid
Skipped fold 80, labels invalid
Skipped fold 89, labels invalid
Skipped fold 145, labels invalid
Skipped fold 165, labels invalid
Skipped fold 178, labels invalid
Skipped fold 204, labels invalid
Skipped fold 214, labels invalid
Skipped fold 229, labels invalid
Skipped fold 356, labels invalid
Skipped fold 427, labels invalid
Skipped fold 428, labels invalid
Skipped fold 562, labels invalid
                                                                                                            precision    recall  f1-score   support

                                 CC: Amplifying Climate Fears: Amplifying existing fears of global warming       0.64      0.71      0.67        45
                                               CC: Amplifying Climate Fears: Doomsday scenarios for humans       0.32      0.54      0.40        13
                                            CC: Amplifying Climate Fears: Earth will be uninhabitable soon       0.50      0.56  

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained on the dom classes:

In [54]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)

y = df[dom_mlb.classes_].values
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=dom_mlb.classes_))

                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.65      0.86      0.74        70
                      CC: Climate change is beneficial       0.00      0.00      0.00         3
              CC: Controversy about green technologies       0.20      0.10      0.13        10
                     CC: Criticism of climate movement       0.45      0.43      0.44        23
                     CC: Criticism of climate policies       0.46      0.71      0.56        38
         CC: Criticism of institutions and authorities       0.51      0.75      0.60        56
                        CC: Downplaying climate change       0.50      0.40      0.44        15
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         2
 CC: Hidden plots by secret schemes of powerful groups       0.50      0.09      0.15        11
          CC: Questioning the measureme

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Lets see if word embeddings get better performance with logistic regression  

Trained on the dom classes:

In [ ]:
X = preprocess(df["text"])
y = df[dom_mlb.classes_].values